<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>

# Exercise: Building your own session-based recommender model

This time it is your turn do some simple coding exercises. You target is to train a session-based recommender model with another SOTA transfomer-based model- `GPT-2`. In doing so replace the `FIXME` lines in the cells with the proper codes based on these instructions:

- Use only product_id-list_seq and 'et_dayofweek_sin-list_seq', 'et_dayofweek_cos-list_seq' columns
- Define the Input module with correct arguments. Keep in mind that GPT-2 supports only causal masking method
- Set number of training epochs to 2
- Define the correct config class for GPT-2 from here
- Disable weight_tying feature

If you need help click on the `...` for the solution.

In [ ]:
import os
import glob
import torch 
import transformers4rec.torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt

In [ ]:
# Define categorical and continuous columns to fed to training model
x_cat_names =  #FIXME
x_cont_names = #FIXME

from merlin_standard_lib import Schema

# Define schema object to pass it to the TabularSequenceFeatures class
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/dli/task/data/")

SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, #FIXME)
schema = Schema().from_proto_text(#FIXME)
schema = schema.select_by_name(#FIXME)

In [ ]:
# Define categorical and continuous columns to fed to training model
x_cat_names =  ['product_id-list_seq']
x_cont_names = ['et_dayofweek_sin-list_seq', 'et_dayofweek_cos-list_seq']

from merlin_standard_lib import Schema

# Define schema object to pass it to the TabularSequenceFeatures class
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/dli/task/data/")

SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(x_cat_names + x_cont_names)

In [ ]:
# Define input block
sequence_length, d_model = 20, 192
# Define input module to process tabular input-features and to prepare masked inputs
inputs= tr.TabularSequenceFeatures.from_schema(
    #FIXME,
    ...
    ...
)

# Define Transformer config class and set default parameters for HF GPT-2 config  
transformer_config = tr.#FIXME.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=sequence_length
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([192]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=#FIXME, hf_format=True, 
                                     metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)

# Get the end-to-end Model class 
model = tr.Model(head)

In [ ]:
# Define input block
sequence_length, d_model = 20, 192
# Define input module to process tabular input-features and to prepare masked inputs
inputs= tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=sequence_length,
    aggregation="concat",
    d_output=d_model,
    masking="causal",
)

# Define XLNetConfig class and set default parameters for HF GPT-2 config  
transformer_config = tr. GPT2Config.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=sequence_length
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([192]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=False, hf_format=True, 
                                     metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)

# Get the end-to-end Model class 
model = tr.Model(head)

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer

#Set arguments for training 
training_args = T4RecTrainingArguments(
            output_dir="./tmp",
            max_sequence_length=20,
            data_loader_engine='nvtabular',
            num_train_epochs=2, 
            dataloader_drop_last=False,
            per_device_train_batch_size = 128,
            per_device_eval_batch_size = 32,
            gradient_accumulation_steps = 1,
            learning_rate=0.000666,
            report_to = [],
            logging_steps=200,
)

In [ ]:
num_train_epochs=2

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True,
)

Train and evaluate the model

In [ ]:
from transformers4rec.torch.utils.examples_utils import fit_and_evaluate
fit_and_evaluate(trainer, start_time_index=1, end_time_index=4, input_dir='/dli/task/data/sessions_by_day')

In [ ]:
input_dir = '/dli/task/data/sessions_by_day'

Congratulations on finishing this notebook. Please execute the cell below to shut down the kernel before moving on to the next notebook, `3-01-Inference-with-Triton.ipynb`.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)